In [1]:
import tensorflow as tf
from keras import layers
from keras import preprocessing
import word2vec as w2v
import dataProsessing as dp
import numpy as np
from tensorboard.plugins import projector
import os
import io

In [2]:

#run this once and only once
os.chdir("../")

In [15]:

%tensorboard --logdir logs\embeddings


UsageError: Line magic function `%tensorboard` not found.


In [ ]:
corpus = dp.loadDataset("Jsb16thSeparated(t_60_rr).json")

In [3]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
embedding_dim = 128
negative_samples = 4
log_dir = "logs\embeddings"

corpus = dp.loadDataset("Jsb16thSeparated(t_60_rr).json")
vocab, inv_vocab = dp.generateVocab(corpus["train"] + corpus["test"]+ corpus["valid"])

vectorizedSequences = dp.vectorizeDataset(corpus["train"],vocab)
targets,contexts,labels = [],[],[]
for s in vectorizedSequences:
    tmp = w2v.generateTrainingData(s,vocab_size=len(vocab),window_size=1,negative_samples=negative_samples)
    targets += tmp[0]
    contexts += tmp[1]
    labels += tmp[2]
    
targets = np.array(targets)
contexts = np.array(contexts)
labels = np.array(labels)

dataset = tf.data.Dataset.from_tensor_slices(((targets,contexts),labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


word2vec = w2v.Word2Vec(len(vocab), embedding_dim,4)
word2vec.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")
word2vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])

weights = tf.Variable(word2vec.get_layer('w2v_embedding').get_weights()[0])

checkpoint = tf.train.Checkpoint(embedding=weights)
checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

dp.generateMetadata(dir = log_dir, inv_vocab = inv_vocab)

out_m = io.open(os.path.join(log_dir, 'metadata.tsv'), 'w', encoding='utf-8')

for i in range(len(inv_vocab.items())):
    out_m.write(str(inv_vocab[i]) + "\n")

out_m.close()


config = projector.ProjectorConfig()
embedding = config.embeddings.add()
# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`.
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings(log_dir, config)




(38424,)
(38424, 5)
(38424, 5)
Epoch 1/20
37/37 [==============================] - 1s 14ms/step - loss: 1.6042 - accuracy: 0.3514
Epoch 2/20
37/37 [==============================] - 1s 14ms/step - loss: 1.5722 - accuracy: 0.8647
Epoch 3/20
37/37 [==============================] - 1s 14ms/step - loss: 1.5115 - accuracy: 0.9696
Epoch 4/20
37/37 [==============================] - 1s 14ms/step - loss: 1.4039 - accuracy: 0.9856
Epoch 5/20
37/37 [==============================] - 1s 14ms/step - loss: 1.2531 - accuracy: 0.9861
Epoch 6/20
37/37 [==============================] - 1s 14ms/step - loss: 1.0832 - accuracy: 0.9846
Epoch 7/20
37/37 [==============================] - 1s 14ms/step - loss: 0.9180 - accuracy: 0.9835
Epoch 8/20
37/37 [==============================] - 1s 14ms/step - loss: 0.7718 - accuracy: 0.9837
Epoch 9/20
37/37 [==============================] - 1s 14ms/step - loss: 0.6471 - accuracy: 0.9847
Epoch 10/20
37/37 [==============================] - 1s 14ms/step - loss: 0.54

TypeError: can only concatenate tuple (not "str") to tuple